In [25]:
import gmsh
import meshio
import pygmsh

# Initialize Circles in 2D Domain (X, Y)

In [26]:
resolution = 0.005
Lx = 10
Ly = 10
radius = 0.90
point0 = (0, 0, 0)
point1 = (Lx, 0, 0)
point2 = (Lx, Ly, 0)
point3 = (0, Ly, 0)

centers = [
           (1, 1, 0),
           (3, 1, 0),
           (5, 1, 0),
           (7, 1, 0),
           (9, 1, 0),
           (2, 2.5, 0),
           (4, 2.5, 0),
           (6, 2.5, 0),
           (8, 2.5, 0),
           (1, 4.10, 0),
           (3, 4.10, 0),
           (5, 4.10, 0),
           (7, 4.10, 0),
           (9, 4.10, 0),
           (1, 6, 0),
           (3, 6, 0),
           (5, 6, 0),
           (7, 6, 0),
           (9, 6, 0),
           (2, 7.5, 0),
           (4, 7.5, 0),
           (6, 7.5, 0),
           (8, 7.5, 0),
           (1, 9.10, 0),
           (3, 9.10, 0),
           (5, 9.10, 0),
           (7, 9.10, 0),
           (9, 9.10, 0),

]
r = radius

# Generate Mesh

In [27]:
geometry = pygmsh.geo.Geometry()
model = geometry.__enter__()

In [28]:
points = [model.add_point(point0, mesh_size=resolution),
          model.add_point(point1, mesh_size=resolution),
          model.add_point(point2, mesh_size=resolution),
          model.add_point(point3, mesh_size=resolution),
          ]

In [29]:
channel_lines = [model.add_line(points[i], points[i+1])
  for i in range(-1, len(points)-1)]
channel_loop = model.add_curve_loop(channel_lines)
circles = [model.add_circle(c, r, mesh_size=resolution) for c in centers]
holes = [circle.curve_loop for circle in circles]
plane_surface = model.add_plane_surface(channel_loop, holes=holes)
model.synchronize()

In [30]:
volume_marker = 6
model.add_physical([plane_surface], "Volume")
model.add_physical([channel_lines[0]], "Left")
model.add_physical([channel_lines[2]], "Right")
model.add_physical([channel_lines[1], channel_lines[3]], "Insulated")
for i, c in enumerate(circles):
    model.add_physical(c.curve_loop.curves, "void%d" % i)

In [31]:
geometry.generate_mesh(dim=2)
gmsh.write("packed-circles.msh")
gmsh.clear()
geometry.__exit__()

In [32]:
mesh_from_file = meshio.read("packed-circles.msh")

In [33]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [34]:
line_mesh = create_mesh(mesh_from_file, "line", prune_z=False)
meshio.write("packed-circles-facets.xdmf", line_mesh)

triangle_mesh = create_mesh(mesh_from_file, "triangle", prune_z=False)
meshio.write("packed-circles.xdmf", triangle_mesh)


# Run Fenics Model

In [35]:
cmd = 'mpirun -n 2 python3 transport_2d.py %d %s %f %d' % (Ly, 'packed-circles', 0.3, Lx)
!{cmd}